In [1]:
%matplotlib_svg
import numpy as np
import pylab as pl
import cv2
from cv2 import cv

## 型態轉換

### 分析cv2的源程式

> **SOURCE**

> `codes\pyopencv_src`：為了方便讀者檢視`cv2`模組的源程式碼，本書提供了自動產生的源程式碼。若讀者遇到參數型態不確定的情況，可以檢視這些檔案中對應的函數。

In [ ]:
%%language c++
static PyObject* pyopencv_line(PyObject* , PyObject* args, PyObject* kw)
{
    PyObject* pyobj_img = NULL;
    Mat img;
    PyObject* pyobj_pt1 = NULL;
    Point pt1;
    PyObject* pyobj_pt2 = NULL;
    Point pt2;
    PyObject* pyobj_color = NULL;
    Scalar color;
    int thickness=1;
    int lineType=8;
    int shift=0;

    const char* keywords[] = { "img", "pt1", "pt2", "color", "thickness", "lineType", "shift", NULL };
    if( PyArg_ParseTupleAndKeywords(args, kw, "OOOO|iii:line", 
                                    (char**)keywords, &pyobj_img, &pyobj_pt1, &pyobj_pt2, 
                                    &pyobj_color, &thickness, &lineType, &shift) &&
        pyopencv_to(pyobj_img, img, ArgInfo("img", 1)) &&
        pyopencv_to(pyobj_pt1, pt1, ArgInfo("pt1", 0)) &&
        pyopencv_to(pyobj_pt2, pt2, ArgInfo("pt2", 0)) &&
        pyopencv_to(pyobj_color, color, ArgInfo("color", 0)) )
    {
        ERRWRAP2( cv::line(img, pt1, pt2, color, thickness, lineType, shift));
        Py_RETURN_NONE;
    }

    return NULL;
}

In [3]:
%%language c++
static int pyopencv_to(const PyObject* o, Mat& m, const ArgInfo info, bool allowND=true);
static inline bool pyopencv_to(PyObject* obj, Point& p, const char* name = "<unknown>");
static bool pyopencv_to(PyObject *o, Scalar& s, const char *name = "<unknown>");

In [4]:
%%language c++
static inline bool pyopencv_to(PyObject* obj, Point& p, const char* name = "<unknown>")
{
    (void)name;
    if(!obj || obj == Py_None)
        return true;
    if(!!PyComplex_CheckExact(obj))
    {
        Py_complex c = PyComplex_AsCComplex(obj);
        p.x = saturate_cast<int>(c.real);
        p.y = saturate_cast<int>(c.imag);
        return true;
    }
    return PyArg_ParseTuple(obj, "ii", &p.x, &p.y) > 0;
}

> **QUESTION**

> 請讀者使用同樣的方法找到與`Scalar`型態對應的`pyopencv_to()`函數，並分析它能將何種型態的物件轉換成`Scalars`物件。

In [23]:
%%language c++
static inline PyObject* pyopencv_from(const RotatedRect& src)
{
    return Py_BuildValue("((ff)(ff)f)", src.center.x, src.center.y, 
                         src.size.width, src.size.height, src.angle);
}

In [26]:
points = np.random.rand(20, 2).astype(np.float32)
(x, y), (w, h), angle = cv2.minAreaRect(points)

### `Mat`物件

In [11]:
cvmat = cv.CreateMat(200, 100, cv2.CV_16UC3)
%C cvmat.height; cvmat.width; cvmat.channels; cvmat.type; cvmat.step

cvmat.height  cvmat.width  cvmat.channels  cvmat.type  cvmat.step
------------  -----------  --------------  ----------  ----------
200           100          3               18          600       


In [12]:
%C cv2.CV_16U; cv2.CV_16UC3

cv2.CV_16U  cv2.CV_16UC3
----------  ------------
2           18          


In [22]:
cv2.normalize(1)

array([[ 1.],
       [ 0.],
       [ 0.],
       [ 0.]])

### 在`cv`和`cv2`之間轉換圖形物件

In [29]:
img = cv2.imread("lena.jpg")
iplimage = cv.LoadImage("lena.jpg")
cvmat = cv.LoadImageM("lena.jpg")
print iplimage
print cvmat

<iplimage(nChannels=3 width=512 height=512 widthStep=1536 )>
<cvmat(type=42424010 8UC3 rows=512 cols=512 step=1536 )>


In [30]:
import numpy as np
np.all(img == np.asarray(iplimage[:]))

True

In [31]:
iplimage2 = cv.GetImage(cv.fromarray(img[::2,::2,:].copy()))